# 모듈, 데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install folium matplotlib mapclassify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 72.7 MB/s eta 0:00:00


In [4]:
import pandas as pd
import geopandas as gpd

In [5]:
df_grid = pd.read_csv('/content/drive/MyDrive/프로젝트1/데이터/unique_grid.csv')
df_grid.head(2)

,gid,lbl,val,geometry,gu
0,다사58a48a,6085.0,6085.0,"POLYGON ((958000 1948000, 958000 1948500, 9585...",강남구
1,다사58b48a,3446.0,3446.0,"POLYGON ((958500 1948000, 958500 1948500, 9590...",강남구


In [6]:
acoustic = pd.read_csv('/content/drive/MyDrive/프로젝트1/데이터/도로시설물/서울시 음향신호기 관련 정보.csv', encoding = 'cp949')
acoustic.head(2)

,음향신호관리번호,지주관리번호,방향 (공통),설치일,교체일,X좌표,Y좌표,제조회사,시설번호,작업구분 (공통),표출구분 (공통),종류,신규정규화ID,상태 (공통),공사관리번호,음향신호관리번호.1,이력ID,위치정보,공사형태 (공통)
0,24-0000010308,02-0000087079,180.0,20000101,20000101,203868.73135,563340.40638,NaN,NaN,1,2,1,4583314,1.0,NaN,24-010308,15036,1.0,001
1,24-0000012524,02-0000081308,0.0,NaN,NaN,207048.64541,553516.56105,NaN,NaN,1,2,1,5343753,1.0,NaN,24-012524,17252,1.0,001


# 데이터 정제
- 컬럼중에서 	음향신호관리번호, X좌표, Y좌표 컬럼만 추출
- 널값 제외하고 값이 존재하는 데이터만 추출

In [7]:
acoustic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24320 entries, 0 to 24319
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   음향신호관리번호    24320 non-null  object 
 1   지주관리번호      24320 non-null  object 
 2   방향 (공통)     19812 non-null  float64
 3   설치일         23015 non-null  object 
 4   교체일         22971 non-null  object 
 5   X좌표         19815 non-null  float64
 6   Y좌표         19815 non-null  float64
 7   제조회사        17562 non-null  object 
 8   시설번호        4702 non-null   object 
 9   작업구분 (공통)   24320 non-null  int64  
 10  표출구분 (공통)   24320 non-null  int64  
 11  종류          24320 non-null  int64  
 12  신규정규화ID     16648 non-null  object 
 13  상태 (공통)     24187 non-null  float64
 14  공사관리번호      22872 non-null  object 
 15  음향신호관리번호.1  24320 non-null  object 
 16  이력ID        24320 non-null  int64  
 17  위치정보        19812 non-null  float64
 18  공사형태 (공통)   22226 non-null  object 
dtypes: float64(5), int64(4), 

In [8]:
acoustic = acoustic.loc[ :, ['음향신호관리번호', 'X좌표' , 'Y좌표']]
#clean_acoustic = acoustic[acoustic.X좌표.isnull() == False ][:10000]

In [9]:
acoustic.info()
acoustic.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24320 entries, 0 to 24319
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   음향신호관리번호  24320 non-null  object 
 1   X좌표       19815 non-null  float64
 2   Y좌표       19815 non-null  float64
dtypes: float64(2), object(1)
memory usage: 570.1+ KB


,음향신호관리번호,X좌표,Y좌표
0,24-0000010308,203868.73135,563340.40638
1,24-0000012524,207048.64541,553516.56105
2,24-0000009173,186493.32909,547928.65368
3,24-0000009363,203783.06422,540680.90727
4,24-0000009445,195057.99497,546032.80196


# pandas ⟶ geopandas

## 격자지도 데이터 프레임
```
기존에 .shp을 로드해서 csv로 저장했기 때문에 'geometry'열이 존재한다.

그러나 csv로 저장하면서 기존에 'geometry'열의 데이터 타입이 문자열로 변경되었다.

따라서 wkt.loads를 이용해서 문자열 타입을 폴리곤 타입으로 바꾸고, geodataframe을 구성할 때 좌표계 정보(epsg:5179)를 주었다.

```

In [10]:
from shapely import wkt

df_grid['geometry'] = df_grid['geometry'].apply(wkt.loads)
geo_grids = gpd.GeoDataFrame(df_grid, crs='epsg:5179')

# 음향신호기 데이터 프레임
```
기존에 .csv 파일이었기 때문에 'geometry'열이 존재하지 않는다.

따라서 geodataframe을 구성할 때 x좌표와 y좌표를 이용하여 'geometry' 컬럼을 만들어준다. 

그 후 to_crs() 함수를 이용해 좌표계를 5179 좌표계로 변경한다.

좌표계 설정 이유 : 좌표계의 종류가 많기 때문에 어떤 종류의 좌표계인지 명시해주어야 함

```

In [11]:
geo_acoustic = gpd.GeoDataFrame(
    acoustic, geometry=gpd.points_from_xy(acoustic['X좌표'], acoustic['Y좌표'])
)
geo_acoustic.set_crs(epsg = 5186, inplace = True)
geo_acoustic = geo_acoustic.to_crs(epsg=5179)

In [16]:
geo_acoustic.explore(tiles='http://mt0.google.com/vt/lyrs=m&hl=ko&x={x}&y={y}&z={z}', attr='Google')

TypeError: ignored

## geopandas 의 sjoin을 사용

- 정확히는 모르지만 공간에 따라 데이터를 join 하는 것 같음
- pandas.join과 유사 ( 다른 점: pandas의 join은 데이터가 같은 것끼리 엮지만 geopandas의 sjoin은 공간이 겹치면 엮는다 )

In [13]:
join_geo_df = gpd.sjoin(geo_acoustic, geo_grids, how='left')
join_geo_df.shape

(24320, 9)

In [14]:
# 피벗테이블 이용해서 격자지도별로 음향신호기 카운트
geo_acoustic_by_grid = join_geo_df.pivot_table(index='gid', values='음향신호관리번호', aggfunc='count')
geo_acoustic_by_grid.head(5)

,음향신호관리번호
gid,
다사38a52b,11
다사38a53a,6
다사38a53b,13
다사38b51b,11
다사38b52a,20


In [15]:
geo_acoustic_by_grid.to_csv('geo_acoustic_by_grid')